In [1]:
# section 1 load all the necessary modules and packages
import glob
import time
import geopandas          as      gpd
import netCDF4            as      nc4
import numpy              as      np
import pandas             as      pd
import xarray             as      xr
from   shapely.geometry   import  Polygon
import matplotlib.pyplot  as      plt
import warnings
import sys
from   shapely.geometry   import  Polygon
import os
import shutil


/Users/shg096/opt/anaconda3/envs/myenv38/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
# read the hydrolakes shapefile
folder_to_save = './shp_output/'
extension = ''


# Check the number of lakes that a segment is interseting with

In [3]:
# read the intersected shapefile of river network and 
shp = gpd.read_file(folder_to_save+'river_lake_intersect'+extension+'.shp')
print('Number of lakes in the intersection:',len(np.unique(shp['b_Hylak_id'])))
m = 0
for i in np.unique(shp['a_seg_id']):
    shp_sub = shp[shp['a_seg_id'] == i] # subset the river segment elements
    if len(np.unique(shp_sub['b_Hylak_id'])) > 1: # segment intersect with more than 1 lake
        print(i) # print the segment ID
        print('number of lakes: ',len(np.unique(shp_sub['b_Hylak_id']))) # print the number of lake
        print('lake ids: ',(np.unique(shp_sub['b_Hylak_id']))) # print lake ID
        print('----')
        m = m + 1 # counter
# num seg intersecting with more than 1 lake
print('number of segments that intersect with more than one lake :', m)



Number of lakes in the intersection: 63888
1001636
number of lakes:  2
lake ids:  [7013787 7013794]
----
1001726
number of lakes:  2
lake ids:  [7128945 8149264]
----
1001939
number of lakes:  3
lake ids:  [8246047 8246254 8246310]
----
1002104
number of lakes:  2
lake ids:  [7015093 8374660]
----
1002130
number of lakes:  2
lake ids:  [7014989 8372838]
----
1002401
number of lakes:  3
lake ids:  [7129434 7129479 8151300]
----
1002546
number of lakes:  2
lake ids:  [7154943 8279777]
----
1002685
number of lakes:  2
lake ids:  [8142978 8143210]
----
1002705
number of lakes:  2
lake ids:  [7168905 7168906]
----
1002727
number of lakes:  2
lake ids:  [8366245 8366250]
----
1002747
number of lakes:  2
lake ids:  [8406214 8406218]
----
1002905
number of lakes:  2
lake ids:  [7174905 8373338]
----
1003130
number of lakes:  2
lake ids:  [8262724 8262773]
----
1003237
number of lakes:  2
lake ids:  [7151182 8264791]
----
1003452
number of lakes:  2
lake ids:  [8317610 8317742]
----
1003506
num

# Simplify the intersection step 1

### Remove the lakes in the river_intersection_lake_arealake that have more than have segments which intersect with them model than needed.

In [4]:
# read the intersected shapefile of river network and 
shp = gpd.read_file(folder_to_save+'river_lake_intersect'+extension+'.shp')
lake_ID = np.unique(shp['b_Hylak_id']) # get the ID of the lakes in teh intersetion
print('Number of lakes in the original intersection:',len(np.unique(shp['b_Hylak_id'])))
shp['remove'] = 0 # initializing the remove field

# looping over the segments, and remove the multiple lakes based on the area
warnings.simplefilter('ignore') # silent the warning
for i in np.unique(shp['a_seg_id']):
    shp_sub = shp[shp['a_seg_id'] == i] # subset based on river segments
    if len(np.unique(shp_sub['b_Hylak_id'])) > 1: # segment intersect with more than 1 lake
        # print(i) # print segment ID
        shp_sub = shp_sub.sort_values(by = ['b_Lake_are'], ascending=False) # sort based on lake area
        max_lake_area = shp_sub['b_Lake_are'].max() # get the maximume lake area
        for index, row in shp_sub.iterrows():
            if shp['b_Lake_are'].loc[index] != max_lake_area:
                shp['remove'].loc[index] = 1 # loop over the subset shapefile and remove lake with smaller area
warnings.simplefilter('default') # back to normal

shp = shp [shp['remove']==1] # remove lakes which are flagged to be removed
shp = shp.sort_values(by= 'a_seg_id') # sort based on seg_id
shp = shp.reset_index(drop=True) # reindex based on the new sorting

#get the lake ID of the removed object
lake_ID_remove = np.unique(shp['b_Hylak_id']) # the IDs that are removed
lake_ID_keep = np.setdiff1d(lake_ID, lake_ID_remove) # the IDs that should be kept

# subset the intersection based on the lake_ID that are kept
shp = gpd.read_file(folder_to_save+'river_lake_intersect'+extension+'.shp') # load the shapefile
shp = shp[shp['b_Hylak_id'].isin(lake_ID_keep)] # subset the shapefile to IDs that are kept
print('Number of lakes in the simplified intersection:',len(np.unique(shp['b_Hylak_id']))) # print the number
shp.to_file(folder_to_save+'river_lake_intersect_simplified'+extension+'.shp') # save to file


Number of lakes in the original intersection: 63888
Number of lakes in the simplified intersection: 25896


# Simplify the intersection step 2
### Remove the lakes that have only on river segment passing through them

In [5]:
shp = gpd.read_file(folder_to_save+'river_lake_intersect_simplified'+extension+'.shp')
print('Number of lakes in the simplified intersection:',len(np.unique(shp['b_Hylak_id'])))
# remove the lakes that have only one river segment (unresolvabale lakes)
shp['keep'] = 1

# looping over lakes and take out the lake that has only one river segment 
warnings.simplefilter('ignore') # silent the warning
for i in np.unique(shp['b_Hylak_id']):
    # print(i)
    shp_sub = shp[shp['b_Hylak_id'] == i] # subset the river segment inside a lake
    if len(np.unique(shp_sub['a_seg_id'])) == 1:
        indx = shp_sub.index
        shp['keep'].loc[indx] = 0
warnings.simplefilter('default') # back to normal

shp = shp [shp['keep']==1]
shp = shp.sort_values(by= 'a_seg_id')
shp = shp.reset_index(drop=True)

print('Number of lakes in the simplified intersection:',len(np.unique(shp['b_Hylak_id'])))
shp.to_file(folder_to_save+'river_lake_intersect_simplified'+extension+'.shp')



Number of lakes in the simplified intersection: 25896
Number of lakes in the simplified intersection: 4236


# Check the simplified result
### To reassure that the simplified intersection has one lake per segment

In [6]:
# read the intersected shapefile of river network and 
shp = gpd.read_file(folder_to_save+'river_lake_intersect_simplified'+extension+'.shp')
print('Number of lakes in the intersection:',len(np.unique(shp['b_Hylak_id'])))
m = 0
for i in np.unique(shp['a_seg_id']):
    shp_sub = shp[shp['a_seg_id'] == i] # subset the river segment elements
    if len(np.unique(shp_sub['b_Hylak_id'])) > 1: # segment intersect with more than 1 lake
        # print(i) # print the segment ID
        print('number of lakes: ',len(np.unique(shp_sub['b_Hylak_id']))) # print the number of lake
        print('lake ids: ',(np.unique(shp_sub['b_Hylak_id']))) # print lake ID
        print('----')
        m = m + 1 # counter
# num seg intersecting with more than 1 lake
print('number of segments that intersect with more than one lake :', m)



Number of lakes in the intersection: 4236
number of segments that intersect with more than one lake : 0


# Correct the length based on what is under the lake
### Here the length is corrected for the segments that have a link with the lake (the simplified version)

In [7]:
shp = gpd.read_file(folder_to_save+'river_lake_intersect_simplified'+extension+'.shp')

# reproect to meter projection and prepare the field to allocated
shp_temp = shp.to_crs ("EPSG:6933") # project to equal area
shp ['LenInLake'] = shp_temp.length
shp ['total_Len'] = 0
shp ['corr_Len'] = 0
shp ['keep'] = 0

warnings.simplefilter('ignore') # silent the warning
for i in np.unique(shp['a_seg_id']):
    # print(i)
    shp_sub = shp[shp['a_seg_id'] == i] # subset the river segment inside a lake
    indx = shp_sub.index
    shp['LenInLake'].loc[indx[0]] = shp_sub['LenInLake'].sum()
    shp_sub_dis = shp_sub.dissolve()
    if len(shp_sub_dis) > 1:
        sys.exit('Dissolve is not resulting in one linear segment')
    shp['geometry'].loc[indx[0]] = shp_sub_dis['geometry'].loc[0]
    shp['keep'].loc[indx[0]] = 1
warnings.simplefilter('default') # back to normal


shp = shp [shp['keep']==1]
shp = shp.sort_values(by= 'a_seg_id')
shp = shp.reset_index(drop=True)

shp_all = gpd.read_file('/Volumes/F:/hydrography/HDMA/processed/hdma_global_stream.shp')
shp_all = shp_all[shp_all['seg_id'].isin(shp['a_seg_id'])]
shp_all = shp_all.to_crs ("EPSG:6933") # project to equal area
shp_all = shp_all.sort_values(by='seg_id')
shp_all = shp_all.reset_index(drop=True)
shp_all ['length'] = shp_all.length
print(len(shp_all))
shp ['total_Len'] = shp_all ['length']
shp ['corr_Len'] = (shp['total_Len']-shp['LenInLake'])
shp ['corr_Len'] = (shp ['corr_Len']/shp['total_Len'])
shp ['corr_Len'] = (shp ['a_Length']*shp ['corr_Len'])


shp.to_file(folder_to_save+'river_lake_intersect_simplified'+extension+'_final.shp')
print(len(shp))



23707


/Users/shg096/opt/anaconda3/envs/myenv38/lib/python3.8/site-packages/geopandas/_vectorized.py:150: DeprecationWarning: An exception was ignored while fetching the attribute `__array_interface__` from an object of type 'MultiLineString'.  With the exception of `AttributeError` NumPy will always raise this exception in the future.  Raise this deprecation warning to see the original exception. (Warning added NumPy 1.21)
  out[:] = [_pygeos_to_shapely(geom) for geom in data]


23707


# identify the outflow and inflow for the river segments


In [8]:
shp = gpd.read_file(folder_to_save+'river_lake_intersect_simplified'+extension+'_final.shp')

shp ['inflow'] = 1
shp ['outflow'] = 0

warnings.simplefilter('ignore') # silent the warning
for i in np.unique(shp['b_Hylak_id']):
    # print(i)
    shp_sub = shp[shp['b_Hylak_id'] == i] # subset the river segment inside a lake
    max_area = shp_sub['a_flow_acc'].max()
    indexs = shp_sub.index
    for index in indexs:
        if (shp_sub['a_flow_acc'].loc[index] == max_area) and (shp_sub['a_Tosegmen'].loc[index] > 0):
            shp ['inflow'].loc[index] = 0
            shp ['outflow'].loc[index] = 1
warnings.simplefilter('default') # silent the warning

shp.to_file(folder_to_save+'river_lake_intersect_simplified'+extension+'_final.shp')



/Users/shg096/opt/anaconda3/envs/myenv38/lib/python3.8/site-packages/geopandas/_vectorized.py:150: DeprecationWarning: An exception was ignored while fetching the attribute `__array_interface__` from an object of type 'MultiLineString'.  With the exception of `AttributeError` NumPy will always raise this exception in the future.  Raise this deprecation warning to see the original exception. (Warning added NumPy 1.21)
  out[:] = [_pygeos_to_shapely(geom) for geom in data]


# update the lake larger than a area for the ones that are selected

In [9]:
# # read the hydrolakes shapefile
shp = gpd.read_file(folder_to_save+'HydroLAKES_polys_v10_file_1'+extension+'.shp')
shp_int = gpd.read_file(folder_to_save+'river_lake_intersect_simplified'+extension+'_final.shp')
shp = shp [shp['Hylak_id'].isin(np.unique(shp_int['b_Hylak_id']))]
shp.to_file(folder_to_save+'HydroLAKES_polys_v10_'+extension+'_final.shp') # save the subset of the shapefile as a new shapefile
shp_buffer = shp.buffer(0.1)
shp_buffer.to_file(folder_to_save+'HydroLAKES_polys_v10_'+extension+'_final_visualize.shp')


/Users/shg096/opt/anaconda3/envs/myenv38/lib/python3.8/site-packages/geopandas/_vectorized.py:150: DeprecationWarning: An exception was ignored while fetching the attribute `__array_interface__` from an object of type 'Polygon'.  With the exception of `AttributeError` NumPy will always raise this exception in the future.  Raise this deprecation warning to see the original exception. (Warning added NumPy 1.21)
  out[:] = [_pygeos_to_shapely(geom) for geom in data]
/var/folders/yh/b1qy7zb96k980mcb2ps9n6d9t1c6zr/T/ipykernel_50952/926084080.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  shp_buffer = shp.buffer(0.1)
